## Continuous White Noise Model

The Equation for Noise discretization is:

$$\mathbf Q = \int_0^{\Delta t} \mathbf F(t)\mathbf{Q_c}\mathbf F^\mathsf{T}(t) dt$$

where: $\mathbf F(t)\mathbf{Q_c}\mathbf F^\mathsf{T}(t)$ is a projection of the noise based on the process model F. Basically, finds how much noise is added per epoch, which is why the equation is integrated in $[0, \Delta t]$

For a Fundemental (state transition) matrix of a newtonian system thats modeling pos,vel and acceleration we can estimate the process noise as follows.

note: this example assumes that the acceleration term is not perfectly modeled, so noise is only applied to that term

In [1]:
import sympy
from sympy import (init_printing, Matrix, MatMul, 
                   integrate, symbols)

init_printing(use_latex='mathjax')

dt, phi = symbols('\Delta{t} \Phi_s')

# Newtonian system modeling pos,vel,accel

F_k = Matrix([[1, dt, dt**2/2],
              [0,  1,      dt],
              [0,  0,       1]])

# phi is spectral density, this parameter is tuned based on sim
# as it is difficult to calculate
Q_c = Matrix([[0, 0, 0],
              [0, 0, 0],
              [0, 0, 1]])*phi

# Continous noise discretization equation
Q = integrate(F_k * Q_c * F_k.T, (dt, 0, dt))

# factor phi out of the matrix to make it more readable
Q = Q / phi
MatMul(Q, phi)

⎡         5           4           3⎤      
⎢\Delta{t}   \Delta{t}   \Delta{t} ⎥      
⎢──────────  ──────────  ──────────⎥      
⎢    20          8           6     ⎥      
⎢                                  ⎥      
⎢         4           3           2⎥      
⎢\Delta{t}   \Delta{t}   \Delta{t} ⎥      
⎢──────────  ──────────  ──────────⎥⋅\Phiₛ
⎢    8           3           2     ⎥      
⎢                                  ⎥      
⎢         3           2            ⎥      
⎢\Delta{t}   \Delta{t}             ⎥      
⎢──────────  ──────────  \Delta{t} ⎥      
⎣    6           2                 ⎦      

 Here are the $Q_c$ for 0th and 1st order newtonian systems:

In [6]:
# only modeling position
F_k = Matrix([[1]])
Q_c = Matrix([[phi]])

print('0th order discrete process noise')
integrate(F_k*Q_c*F_k.T,(dt, 0, dt))

0th order discrete process noise


[\Delta{t}⋅\Phiₛ]

In [7]:
# modeling position and velocity
F_k = Matrix([[1, dt],
              [0, 1]])

Q_c = Matrix([[0, 0],
              [0, 1]]) * phi

Q = integrate(F_k * Q_c * F_k.T, (dt, 0, dt))

print('1st order discrete process noise')
# factor phi out of the matrix to make it more readable
Q = Q / phi
MatMul(Q, phi)

1st order discrete process noise


⎡         3           2⎤      
⎢\Delta{t}   \Delta{t} ⎥      
⎢──────────  ──────────⎥      
⎢    3           2     ⎥      
⎢                      ⎥⋅\Phiₛ
⎢         2            ⎥      
⎢\Delta{t}             ⎥      
⎢──────────  \Delta{t} ⎥      
⎣    2                 ⎦      

## Piecewise Noise Model

The Advantage of this model to the continuous model, is that we can model the noise in terms of $\sigma^2$, which we can describe in terms of the motion and amount of error that we expect.
- this is a bit more intuitive compared to the first method, but the continuous method handles time-varying samples better because the noise is integrated across the time period.